In [4]:
'''
Merge athletes who were not found on the team page that match an 
existing athlete in the DB.
'''

import pandas as pd
from util.db_util import Database

db = Database("db/Track.db")

# get athletes that have first = last and grad_year = 9999
# if we don't find an athlete when web scrapping, we put the full
# name in for both first and last
df = db.get_problem_athletes()

results = []

for index, row in df.iterrows():
    full_name = row["first"].strip().split()
    
    if len(full_name) >= 2:
        first = full_name[0]
        last = " ".join(full_name[1:])
        
        school_id = row["school_id"]
    
        bad_id = row["athlete_id"]

        # check for matching athlete 
        good_id = db.get_athlete_id_wo_grad_year(first, last, school_id)

        if good_id is not None:
            athlete_df = db.get_athlete(good_id)
            results.append((good_id, bad_id))
    else:
        print("BAD")
        print(full_name)

results_df = pd.DataFrame(results, columns=["Good", "Bad"])
print(results_df)

file = f"Merge ID Results.csv"
results_df.to_csv(file, index=False)


BAD
['III']
      Good     Bad
0   302658  292349
1   302877  292818
2   303570  294230
3   303577  294257
4   302003  294808
..     ...     ...
62  303254  312881
63  303243  312882
64  303431  312883
65  303407  312884
66  303481  312885

[67 rows x 2 columns]


In [7]:
for index, row in results_df.iterrows():
    good_id, bad_id = int(row["Good"]), int(row["Bad"])
    db.merge_athlete(good_id, bad_id)


In [19]:
'''
Merge athletes who are in the database with the same first, last,
school_id but different grad_years (within 4 years of each other). In 
this case, keep the athlete_id with the highest grad_year. 
'''

import pandas as pd
from util.db_util import Database

db = Database("db/Track.db")

# 1. Load all athletes
df = pd.read_sql_query("""
    SELECT athlete_id, first, last, gender, school_id, grad_year
    FROM athlete
""", db.conn)

# 2. Group by identity fields
groups = df.groupby(["first", "last", "gender", "school_id"])

merge_count = 0

for _, group in groups:
    if len(group) <= 1:
        continue

    # 3. Pick athlete with highest grad_year to keep
    keep_row = group.loc[group["grad_year"].idxmax()]
    keep_id = int(keep_row["athlete_id"])

    # 4. Merge all others into keep_id
    for _, row in group.iterrows():
        bad_id = int(row["athlete_id"])

        if (bad_id == keep_id):
            continue

        if int(keep_row["grad_year"]) == 9999:
            new_bad_id = keep_id
            keep_id = bad_id
            bad_id = new_bad_id
        elif abs(int(keep_row["grad_year"]) - int(row["grad_year"])) > 4:
            continue

        print(keep_id, bad_id)
        db.merge_athlete(keep_id, bad_id)
        merge_count += 1

print(f"Done. Merged {merge_count} duplicate athlete records.")


Done. Merged 0 duplicate athlete records.


In [2]:
from util.db_util import Database

db = Database("db/Track.db")

db.merge_athlete(301208, 301209)